In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_WyRytb0.csv', 'train-scene classification']


In [2]:
import os
import re
import cv2
import glob
import shutil
import random
import numpy as np
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img


TRAIN_DIR = '../input/train-scene classification/train/'
TEST_DIR = '../input/test_WyRytb0.csv/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

y_test = pd.read_csv("../input/test_WyRytb0.csv")
y_train = pd.read_csv("../input/train-scene classification/train.csv")
y_train = y_train['label']
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, 6)

train_images_list = []
test_images_list = []
a = list(y_test['image_name'])
for i in train_images:
    if str(i[42:]) not in a:
        train_images_list.append(i)
    if str(i[42:]) in a:
        test_images_list.append(i)
        
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_images_list.sort(key=natural_keys)
test_images_list.sort(key=natural_keys)

def prepare_data(list_of_images):
    x = [] 
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (150,150), interpolation=cv2.INTER_CUBIC))     
    return x
X_train = prepare_data(train_images_list)
X_test = prepare_data(test_images_list)



Using TensorFlow backend.


In [3]:
X_train_prime, X_val, y_train_prime, y_val = train_test_split(X_train, y_train_categorical, random_state = 6, train_size = .75)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow(np.array(X_train_prime), y_train_prime, batch_size = 16, shuffle = True)
validation_generator = val_datagen.flow(np.array(X_val), y_val, batch_size = 16, shuffle = True)


from keras.applications.vgg16 import VGG16
vgg_conv = VGG16(weights = 'imagenet',include_top = False,input_shape = (150,150,3))


import tensorflow as tf
print(tf.__version__)

for layer in vgg_conv.layers[:8]:
    layer.trainable = True

for layer in vgg_conv.layers:
    print(layer,layer.trainable)
    
    
from keras import models,layers,optimizers
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024,  activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(6,activation = 'softmax'))

#model.summary()
model.compile(loss='categorical_crossentropy',optimizer = optimizers.Adam(lr=1e-5),metrics=['acc'])

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


58892288/58889256 [==============================] - 1s 0us/step
1.12.0
<keras.engine.input_layer.InputLayer object at 0x7fa9126a7550> True
<keras.layers.convolutional.Conv2D object at 0x7fa9126a7a90> True
<keras.layers.convolutional.Conv2D object at 0x7fa9126a7940> True
<keras.layers.pooling.MaxPooling2D object at 0x7fa91278c240> True
<keras.layers.convolutional.Conv2D object at 0x7fa91278c358> True
<keras.layers.convolutional.Conv2D object at 0x7fa912903390> True
<keras.layers.pooling.MaxPooling2D object at 0x7fa91286f828> True
<keras.layers.convolutional.Conv2D object at 0x7fa91286f6a0> True
<keras.layers.convolutional.Conv2D object at 0x7fa912890128> True
<keras.layers.convolutional.Conv2D object at 0x7fa912c5a208> True
<keras.layers.pooling.MaxPooling2D object at 0x7fa912c7d828> True
<keras.layers.convolutional.Conv2D object at 0x7fa912c7d6a0> True
<keras.layers.convolutional.Conv2D object at 0x7fa91296e128> True
<keras.layers.convolutional.Conv2D object at 0x7fa912cf7208> True
<k

In [4]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch = 1000,
    epochs = 3,
    shuffle = True,
    validation_data = validation_generator,
    validation_steps = 500,
    use_multiprocessing=True
)


Epoch 1/3
1000/1000 [==============================] - 349s 349ms/step - loss: 0.3906 - acc: 0.8606 - val_loss: 0.2587 - val_acc: 0.9053
Epoch 2/3
1000/1000 [==============================] - 340s 340ms/step - loss: 0.2273 - acc: 0.9195 - val_loss: 0.2399 - val_acc: 0.9140
Epoch 3/3
1000/1000 [==============================] - 338s 338ms/step - loss: 0.1732 - acc: 0.9389 - val_loss: 0.2006 - val_acc: 0.9259


In [5]:
X_test = np.array(X_test)/255.0
prediction_probabilities = model.predict(X_test)
counter = range(1, len(test_images_list) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']
for i, j in enumerate(solution['label']):
    if(i%1000 == 0):
        print(i+1)
    solution['label'][i] = np.argmax(j)
y_test['label'] = solution['label']
y_test.to_csv("Scene1.csv", index = False)

1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1001
2001
3001
4001
5001
6001
7001
